\section{Ben}

```python
def Test(Test)
  if not Test:
    return Test
```

This is a text.  
This is another text.

# Variability inter NNs 

Neural networks are by default variables do not produce the same output from one NN trained to the next even with the same hyperparameter. A few parameters can be set in order to get reproducible NNs :
1. Using the same training/validation datasets helps as seeing different values would lead to different optimization steps.
2. Setting a seed number for the weight initilization  
```python
        tf.random.set_seed(seed)
```
This function allows tensorflow to initilised the same weights run after run as long as the NN neuron architecture stays the same.
3. Tensorflow performs a shuffling before training and inbetween epochs, a certain parameter can be set to remove said shuffling, maybe *tf.random.set_seed* works on this shuffling (?). Further test is needed.

Some variability is expected between NN runs, NN sometimes specializes in certain scenarios. Exemple in my report Table 8 with NN added slope and bathymetry, a very small RMSE was obtained for Ocean4. Either this NN had initialised weights that were better suited for Ocean4 or the NN saw more Ocean4 points in batchs at the start of the optimisation. The idea behind it as far as I know is that different NN have a certain probability of producing very good parameterisation once trained. Meaning that even simpler configuration could, given enough re run produce a very good NN after training if by chances the most optimal structure in the NN are predisposed in the initilisation phase. The probability of getting such NN should therefore increase when adding new variables and increasong the NN architecture complexity (added neurons, lr ...)

# NN hyperparameter

## Neuron architecture

The NN is constructed using a input layer and stacked fully connected(dense) layers, each dense layer composed of neuron performing a weighted sum of its inputs using its optimised weights. 
Though the neuron benchmark, it was found that deeper and more complex do not provide sufficient enough increase in precision to justify their high training cost. Especially simpler inputs such as (T, S, iceD) which we had.



## Activation function

The activation fucntion is what gives the possibility for the NN to learn and reproduce non linear relationships. We chose here to use a "swish" activation function compared to the more popular "Relu" function. Tests were conducted and swish NN were found to be 10-12% better than their Relu counterparts, the trending also seemed to be that this difference is supposed to widen as we increase the NN architecture complexity.  
It is though undeniable that Relu networks should compute faster than the Swish NN, considering that a big NN (128X5) on the biggest datasets (~3-4 millions points) takes about 20 seconds (using swish), I do not think it is such a big deal.

## Learning rate


Multiple learning rate scheduling method were explored:
1. Divided by 5 every 8 epochs
2. Divided by 2 every 10 epochs 
3. Method plateau 

First observation:  
Whenever no minimum lr is introduced to method 1., since the method very rapidly descend in lr scale. At the end of the 2nd transition 16th epochs, the NN does not train anymore due to lr too low.

First motivation:
The lr scheduling was first introduced on a less advanced variable input dataset (T(z), S(z), iceD, Dist grounding)(following second method of expolation 40 T and 40 S), due to what I referred to as an "overshoot" pattern on the training curves, where the NN seems to bounce around (see below).

The first implementation followed method 1., the produced NN divided the val mse by 2 compared to normal setups. Overall RMSE divided by 2 aswell, main benefit seen in the representation of Ocean4 notoriously hard to reproduce due to its small melt. Introducing this method does not increase training time and is therefore only a bonus.

## Batch size

The batch size dictates how many sample points are used for every optimization steps, it is common knowledge increasing the batch size generally leads to quicker training time at the cost of precision. However some strange behaviors were observed where NNs trained using a 1024 or 2048 batch size consistently outperformed similarly trained NN using 64 and 128 batch size.
However using 128 or 1024 alongside any form of lr scheduling does bring all NN together. Increasing the batch size seems to have a similar effect to the adding of lr schedule.
Further goals:
- Possible batch size schedulling (ref https://arxiv.org/abs/1711.00489v2).
- Possible to study the generalization potential for higher size of batches.

Study went as far as $(2-3)*10^{4}$ batch size during their last scheduling step.

[//]: ![batch](Secret_images/Effect_of_batch_size.png)
<img src="Secret_images/Effect_of_batch_size.png" width="500" height="340">
Training curves of val_mse for different batch size usage

## Dropout

Dropout out layers works by disabling connections between neurones, the disabling happens at every batch end using a probability given to the function. The previously disabled is then re enabled and can be disabled again at the next dropout drawing. This method is supposed to spread learning and further train networks showing overfitting symptom. 

Multiple setup explored  
1. Using dropout with increased learning rate(lr) (x10-100) consistently produced non converging neural networks.
2. Using dropout with a decreased lr through any sort of lr scheduling produced worse but still converging NNs. Plateau'ed mse generally two times higher than non dropout variant.
3. Using dropout without any kind of lr scheduling, results matched the output of the non hybrid non dropout variant in terms of mse and val_mse.  
4. Establishing a hybrid method with training executed in two parts, first part with dropout enabled. Then after a certain number of epochs switch to second part, where dropout layers are removed and lr scheduling introduced. The first part is supposed to create a "stable architecture" that could be trained further in the second part with the lr scheduling. Results matched the output of the non hybrid non dropout variant in terms of mse and val_mse.  

All were tested with various dropout probability from 0-50% chance of being dropped.

Further goals : 
- Test the generalisation potential of both standard and hybrid dropout method in general. Maybe more precisely on the composite datasets. Even if the obtained mse is higher, the produced NN might be more generalizable.
- Possible use of both dropout and pruning method (?)

# Generalisation potential

As what was discussed in the oral presentation's discussion, the poor generalization potential might be due to overfitting. However for this case, the plots presented did had quite different conditions (calving being the main one). But definetely, is the generalisation problem encountered for the composite due to overfitting (?)




#### Potential solutions

Based on the observations that a 96X5(96-96-96-96-96) composite trained NN performed better on outside scenarios compared to a 128X5 NN. A decrease of the NN complexity might be helpful when creating a more general NN, multiple possible ways can be explored:
1. A decreased in network complexity, through less neurons and less layers. Or keeping the large architecture and introducing either pruning or potential dropout. 
2. Maybe disabling the lr scheduling, as the "small details" learned by the NN might not be generalizable relationships. 
3. Potential in exploring the use of either increased batch size 4096-8192 or a batch size scheduling method. 
4. A decreased in input complexity, might be interesting to see the generalisability of a NN trained only on T, S, iceDraft with both extrapolation method explored. 
5. Might be useful to use some sort of dropout technique here either simple or hybrid method. If the composite NN did overfit, this method might re calibrate it (?).

First results in increasing the potential generalization :
The simple constant pruning does not seem to increase the NN accuracy on the non trained scenarios. What can be observed however is the variability inter NN trained, where no pattern can be inferred but random NN seem to perform good.  

Decreasing the NN complexity by reducing its size 

# Interesting outlook for further testing

A desire of mine when starting up was to developpe a fully offline (/ unsupervised) NN, meaning the data would not have to be produced by a coupled ice sheet ocean model (Heavy to run). Especially when considering the speed of the NN, this could be something spectacular.  
To do so I believe it is possible to make small steps in this direction:
1. Instead of our only one output (melt rate) for a given point, it is possible to predict another variable such as the iceDraft at the next time increment. This would not only make the NN emulate NEMO's melt rate (same as before) outputs but also Elmer Ice's new ice topography computations. 
2. In a second time, it would be interesting to use GAN (Generative Adversarial Networks) to learn how to reproduce realistic temperature/salinity following possible scenarios flags in the training process to make different type of forcings.
3. The produced NN and GAN could then be used fully offline to where the GAN gives the forcing temperature near the cavity, the NN produces melt rates and its new ice topography for the next time step.
4. From there more complex relationships could be programmed to where melt water could be taken into account for the generation of the new temperature/salinity by the GAN.
5. Say all of this was possible with the GAN's speed being somewhat similar to the NN (hypothetically), it could be expected to produce using this fully offline method, a 100 years of simulation time (~3-4 millions points) in a minute or less. It would probably be a bit more as the new ice slopes, dist (ground/front) line need to be computed at every time steps.

List benchmark name of NN:
- Prune_benchmark_ISOMIP, Goal being to study the effect of pruning of a ISOMIP+ trained NN applied to ISOMIP+. Trained for 64 epochs, plateau lr applied and 128X5 neuron architecture. All NN used different sparsity fraction, with the use of the constant sparsity function.
- Prune_benchmark_Composite, Goal being to study the effect of pruning of a Composite trained NN applied to the 2 other non trained MISOMIP scenarios. Trained for 64 epochs, plateau lr applied and 128X5 neuron architecture. All NN used different sparsity fraction, with the use of the constant sparsity function.
- Benchmark swish_relu (50 epochs lr method 1 (Div by 5 every 8 no min lr introduced), main goal compare swish and relu outputs and their respective training time)
- Drop_Var_bench_128 Benchmark done to study the variables importance by not training it. Trained for 128 epochs on the 128X5 architecture using the lr plateau method.
- Complexity_downgrade_composite Trained for 64 epochs on the Composite dataset, plateau lr applied on either 32_32_96_96 or 96X5 neuron architecture. Main goal being to compare their outputs on the 2 remaining MISOMIP datasets compared to the big 128X5 architecture to test if less complex NN might be more generalizable.